# 🤖 Predictive Model Performance
## How do you decide which predictive model to use?
In this notebook, we evaluate several machine learning models to predict whether a plant has medicinal properties based on its taxonomy and other descriptive features. Below is a brief overview of the models used and how they conceptually approach the classification task:

### 1. **Logistic Regression**
- **Type**: Linear model
- **Concept**: Estimates the probability that a plant is medicinal using a weighted combination of input features.
- **Strengths**: Simple, interpretable, fast.
- **Limitations**: Assumes a linear relationship between features and the log-odds of the outcome; struggles with complex patterns.

### 2. **Decision Tree**
- **Type**: Non-linear, rule-based
- **Concept**: Splits the data into branches based on feature thresholds to arrive at a prediction at the leaves.
- **Strengths**: Easy to visualize and understand; captures non-linear relationships.
- **Limitations**: Can overfit the training data if not pruned or regularized.

### 3. **Random Forest**
- **Type**: Ensemble (of decision trees)
- **Concept**: Trains multiple decision trees on different subsets of the data and averages their predictions to reduce variance.
- **Strengths**: More accurate and robust than a single tree; reduces overfitting.
- **Limitations**: Less interpretable; slower than simpler models.

### 4. **Gradient Boosting (e.g., GBT)**
- **Type**: Ensemble (boosted decision trees)
- **Concept**: Trains trees sequentially, where each tree corrects the errors of the previous one using gradient descent.
- **Strengths**: High accuracy; handles complex patterns well.
- **Limitations**: Can overfit if not tuned properly; computationally intensive.

### 5. **Support Vector Machine (SVM)**
- **Type**: Maximum-margin classifier
- **Concept**: Finds the optimal boundary (hyperplane) that best separates medicinal from non-medicinal plants by maximizing the margin between classes.
- **Strengths**: Works well in high-dimensional spaces; robust to overfitting.
- **Limitations**: Not ideal for large datasets; performance depends on kernel choice.

### 6. **XGBoost**
- **Type**: Gradient-boosted tree ensemble (optimized)
- **Concept**: An efficient and regularized implementation of gradient boosting that adds boosting trees iteratively to correct previous errors.
- **Strengths**: Often state-of-the-art in structured data problems; fast and scalable.
- **Limitations**: Complex; requires tuning; less interpretable.

---

Each model captures different aspects of the underlying patterns in the data. By comparing their performance across different evaluation strategies (e.g., with SMOTE and downsampling), we aim to identify not only which models are accurate, but also which ones are most robust under real-world conditions.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from collections import Counter
from xgboost import XGBClassifier


import warnings
warnings.filterwarnings('ignore')

In [2]:
#####################################
# DATA LOADING
#####################################
#Load the pfaf_plants_merged.csv file
df = pd.read_csv('pfaf_plants_merged.csv')
df.head()

,use_keyword,latin_name_search,common_name_search,edibility_rating_search,medicinal_rating_search,plant_url,Care Requirements,Common Name,Common Names,Cultivation Details,...,Native Range,Other Uses,Other Uses Rating,Propagation,Range,Scientific Name,Special Uses,Summary,USDA hardiness,Weed Potential
0,Stomachic,Abelmoschus moschatus,"Musk Mallow,Musk Okra",2,3,https://pfaf.org/user/Plant.aspx?LatinName=Abe...,Moist Soil; Half Hardy; Full sun,"Musk Mallow,Musk Okra","Musk Mallow,Musk Okra",Easily grown in a rich well-drained soil in a ...,...,"TEMPERATE ASIA: China (Hunan Sheng, Jiangxi Sh...",Essential Fibre Insecticide Oil Size An essent...,3.0,Seed - sow April in a greenhouse. The seed ger...,S.E. Asia - Himalayas to China and Vietnam.,Abelmoschus moschatus - Medik.,Scented Plants,NaN,8-11,No
1,Stomachic,Abies grandis,"Grand Fir, Giant Fir, Lowland White Fir",2,2,https://pfaf.org/user/Plant.aspx?LatinName=Abi...,Semi-shade; Fully Hardy; Moist Soil; Full shad...,"Grand Fir, Giant Fir, Lowland White Fir","Grand Fir, Giant Fir, Lowland White Fir","Landscape Uses:Screen, Specimen. Prefers a goo...",...,"NORTHERN AMERICA: Canada (British Columbia), U...",Baby care Dye Gum Incense Repellent Roofing Wo...,3.0,Seed - sow early February in a greenhouse or o...,Western N. America - British Columbia to Calif...,Abies grandis - (Douglas. ex D.Don.)Lindl.,Food Forest Scented Plants,"Form: Columnar, Upright or erect.",5-6,No
2,Stomachic,Abies spectabilis,Himalayan Fir,0,2,https://pfaf.org/user/Plant.aspx?LatinName=Abi...,Semi-shade; Fully Hardy; Moist Soil; Full shad...,Himalayan Fir,Himalayan Fir,Prefers a good moist but not water-logged soil...,...,"TEMPERATE ASIA: Afghanistan, China (Xizang Ziz...",Essential Fuel Incense Wood An essential oil i...,3.0,Seed - sow early February in a greenhouse or o...,E. Asia - Himalayas from Afghanistan to Nepal.,Abies spectabilis - (D.Don.)Spach.,Scented Plants,NaN,6-9,No
3,Stomachic,Abutilon theophrasti,"China Jute, Velvetleaf, Butterprint Buttonweed...",3,2,https://pfaf.org/user/Plant.aspx?LatinName=Abu...,Semi-shade; Fully Hardy; Well drained soil; Mo...,"China Jute, Velvetleaf, Butterprint Buttonweed...","China Jute, Velvetleaf, Butterprint Buttonweed...",Requires full sun or part day shade and a fert...,...,"TEMPERATE ASIA: Afghanistan, Egypt (Sinai), Ir...",Fibre Oil Paper A fibre obtained from the stem...,4.0,Seed - sow early April in a greenhouse. Germin...,Asia - tropical. Naturalised in S.E. Europe an...,Abutilon theophrasti - Medik.,NaN,Form: Upright or erect.,Coming soon,Yes
4,Stomachic,Acacia farnesiana,"Sweet Acacia, Perfume Acacia, Huisache",2,2,https://pfaf.org/user/Plant.aspx?LatinName=Aca...,Moist Soil; Frost Hardy; Full sun; Well draine...,"Sweet Acacia, Perfume Acacia, Huisache","Sweet Acacia, Perfume Acacia, Huisache",Landscape Uses: Pest tolerant. Originally trop...,...,NORTHERN AMERICA: United States (Florida (nort...,Adhesive Dye Essential Gum Gum Ink Tannin Teet...,4.0,Seed - best sown as soon as it is ripe in a su...,"The original range is uncertain, but is probab...",Acacia farnesiana - (L.)Willd.,Nitrogen Fixer Scented Plants,Bloom Color: Yellow. Main Bloom Time: Early su...,9-11,Yes


In [3]:
#####################################
# DATA PREPROCESSING
#####################################
# Drop rows with missing target or taxonomy info
df = df.dropna(subset=['Family', 'Scientific Name', 'medicinal_rating_search'])
# Clean up column names to remove trailing/leading spaces
df.columns = df.columns.str.strip()

# Create a binary target: 1 if medicinal rating > 0, else 0
df['is_medicinal'] = (df['medicinal_rating_search'] > 0).astype(int)
df = df.rename(columns={
    'Edibility Rating': 'edibility',
    'Other Uses Rating': 'other_uses'
})

# Choose features
features = ['Family', 'edibility', 'other_uses']
df = df.dropna(subset=features)

# Encode categorical 'Family'
df['Family'] = LabelEncoder().fit_transform(df['Family'])

# Balance entire dataset (NOT common, but useful for testing model behavior)
X = df[['Family', 'edibility', 'other_uses']]
y = df['is_medicinal']

sm = SMOTE(random_state=42) # crucial due to the extreme class imabalance in the dataset
X_resampled, y_resampled = sm.fit_resample(X, y)

# Now split the fully balanced dataset
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [4]:
#####################################
# MODEL TRAINING AND EVALUATION
#####################################
# Set seed for reproducibility
np.random.seed(42)

# --- Optional: Apply SMOTE to training set ---
use_smote = True  # Set to True because of the class imbalance in the dataset

if use_smote:
    print("Before SMOTE:", Counter(y_train))
    sm = SMOTE(random_state=42)
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
    print("After SMOTE:", Counter(y_train_res))
    scale_pos_weight = 1  # Not used in this case
else:
    X_train_res, y_train_res = X_train, y_train
    # Compute scale_pos_weight for XGBoost (used when not using SMOTE)
    neg, pos = np.bincount(y_train)
    scale_pos_weight = neg / pos

# Define models with class_weight or equivalent
models = {
    'Logistic Regression': LogisticRegression(class_weight='balanced'),
    'Decision Tree': DecisionTreeClassifier(class_weight='balanced'),
    'Random Forest': RandomForestClassifier(class_weight='balanced'),
    'Gradient Boosting': GradientBoostingClassifier(),  # Cannot set class_weight
    'SVM': SVC(probability=True, class_weight='balanced'),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=scale_pos_weight)
}

results = {}

for name, model in models.items():
    model.fit(X_train_res, y_train_res)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
    results[name] = {
        'model': model,
        'pred': y_pred,
        'proba': y_proba,
        'report': classification_report(y_test, y_pred, output_dict=True)
    }
    print(f"=== {name} ===")
    print(classification_report(y_test, y_pred))

df_smote_results = results

Before SMOTE: Counter({0: 11396, 1: 11396})
After SMOTE: Counter({0: 11396, 1: 11396})
=== Logistic Regression ===
              precision    recall  f1-score   support

           0       0.58      0.59      0.59      2849
           1       0.59      0.58      0.58      2849

    accuracy                           0.58      5698
   macro avg       0.58      0.58      0.58      5698
weighted avg       0.58      0.58      0.58      5698

=== Decision Tree ===
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2849
           1       1.00      0.98      0.99      2849

    accuracy                           0.99      5698
   macro avg       0.99      0.99      0.99      5698
weighted avg       0.99      0.99      0.99      5698

=== Random Forest ===
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2849
           1       1.00      0.98      0.99      2849

    accuracy           

## 📝 Evaluation Summary: Model Performance on SMOTE-Augmented Data

To explore the predictive capacity of various models, we applied **SMOTE to the full dataset**, balancing the number of medicinal (1) and non-medicinal (0) plant entries. We then split this balanced data into training and test sets.

### Key Observations:
- **Ensemble tree models** (Random Forest, XGBoost, Decision Tree) achieved near-perfect scores (~99% accuracy), indicating strong capacity to fit the synthetic data.
- **Logistic Regression** and **SVM** performed noticeably worse (~58–64% accuracy), suggesting the need for non-linear decision boundaries.
- Since both training and test sets included synthetic (SMOTEd) samples, the high scores **reflect memorization rather than generalization**.

### ⚠️ Important Caveat:
This evaluation setup does **not represent real-world performance**, as the test set includes synthetic data. For realistic evaluation, SMOTE should be applied **only to the training set**, while the test set remains untouched and imbalanced.

In [5]:
#####################################
# TESTING FOR REAL-WORLD PERFORMANCE
#####################################
# Drop rows with missing target or taxonomy info
df = df.dropna(subset=['Family', 'Scientific Name', 'medicinal_rating_search'])
# Clean up column names to remove trailing/leading spaces
df.columns = df.columns.str.strip()

# Create a binary target: 1 if medicinal rating > 0, else 0
df['is_medicinal'] = (df['medicinal_rating_search'] > 0).astype(int)
df = df.rename(columns={
    'Edibility Rating': 'edibility',
    'Other Uses Rating': 'other_uses'
})

# Choose features
features = ['Family', 'edibility', 'other_uses']
df = df.dropna(subset=features)

# Encode categorical 'Family'
df['Family'] = LabelEncoder().fit_transform(df['Family'])

# Downsample class 1 to match class 0
df_0 = df[df['is_medicinal'] == 0]
df_1 = df[df['is_medicinal'] == 1].sample(n=len(df_0), random_state=42)

df_balanced = pd.concat([df_0, df_1])
X = df_balanced[['Family', 'edibility', 'other_uses']]
y = df_balanced['is_medicinal']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

In [6]:
#####################################
# MODEL TRAINING AND EVALUATION
#####################################
# Set seed for reproducibility
np.random.seed(42)
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from collections import Counter
from xgboost import XGBClassifier

# --- Optional: Apply SMOTE to training set ---
use_smote = True  # Set to True because of the class imbalance in the dataset

if use_smote:
    print("Before SMOTE:", Counter(y_train))
    sm = SMOTE(random_state=42)
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
    print("After SMOTE:", Counter(y_train_res))
    scale_pos_weight = 1  # Not used in this case
else:
    X_train_res, y_train_res = X_train, y_train
    # Compute scale_pos_weight for XGBoost (used when not using SMOTE)
    neg, pos = np.bincount(y_train)
    scale_pos_weight = neg / pos

# Define models with class_weight or equivalent
models = {
    'Logistic Regression': LogisticRegression(class_weight='balanced'),
    'Decision Tree': DecisionTreeClassifier(class_weight='balanced'),
    'Random Forest': RandomForestClassifier(class_weight='balanced'),
    'Gradient Boosting': GradientBoostingClassifier(),  # Cannot set class_weight
    'SVM': SVC(probability=True, class_weight='balanced'),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=scale_pos_weight)
}

results = {}

for name, model in models.items():
    model.fit(X_train_res, y_train_res)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
    results[name] = {
        'model': model,
        'pred': y_pred,
        'proba': y_proba,
        'report': classification_report(y_test, y_pred, output_dict=True)
    }
    print(f"=== {name} ===")
    print(classification_report(y_test, y_pred))

df_downsampled_results = results

Before SMOTE: Counter({1: 30, 0: 30})
After SMOTE: Counter({1: 30, 0: 30})
=== Logistic Regression ===
              precision    recall  f1-score   support

           0       0.50      0.38      0.43         8
           1       0.50      0.62      0.56         8

    accuracy                           0.50        16
   macro avg       0.50      0.50      0.49        16
weighted avg       0.50      0.50      0.49        16

=== Decision Tree ===
              precision    recall  f1-score   support

           0       0.33      0.12      0.18         8
           1       0.46      0.75      0.57         8

    accuracy                           0.44        16
   macro avg       0.40      0.44      0.38        16
weighted avg       0.40      0.44      0.38        16

=== Random Forest ===
              precision    recall  f1-score   support

           0       0.57      0.50      0.53         8
           1       0.56      0.62      0.59         8

    accuracy                       

In [7]:
#####################################
# VISUALIZATION AND INTERACTIVE WIDGETS
#####################################
def extract_classification_metrics(results_dict, eval_set_name):
    rows = []
    for model_name, result in results_dict.items():
        report = result['report']
        for cls in ['0', '1']:
            rows.append({
                'Model': model_name,
                'Metric': 'F1',
                'Class': int(cls),
                'Value': report[cls]['f1-score'],
                'Set': eval_set_name
            })
            rows.append({
                'Model': model_name,
                'Metric': 'Precision',
                'Class': int(cls),
                'Value': report[cls]['precision'],
                'Set': eval_set_name
            })
            rows.append({
                'Model': model_name,
                'Metric': 'Recall',
                'Class': int(cls),
                'Value': report[cls]['recall'],
                'Set': eval_set_name
            })
    return pd.DataFrame(rows)

# 1. Use your real results dictionaries (from your model evaluation)
smote_results = {k: v for k, v in df_smote_results.items() if k != 'Set'}
downsampled_results = {k: v for k, v in df_downsampled_results.items() if k != 'Set'}

# 2. Extract performance metrics into tidy DataFrames
df_smote_metrics = extract_classification_metrics(smote_results, 'SMOTE')
df_downsampled_metrics = extract_classification_metrics(downsampled_results, 'Downsampled')

# 3. Combine them for plotting
df_viz = pd.concat([df_smote_metrics, df_downsampled_metrics], ignore_index=True)

# Create widgets
metric_dropdown = widgets.Dropdown(
    options=['F1', 'Precision'],
    value='F1',
    description='Metric:'
)

class_dropdown = widgets.Dropdown(
    options=[0, 1],
    value=1,
    description='Class:'
)

def update_plot(metric, target_class):
    filtered = df_viz[
        (df_viz['Metric'] == metric) &
        (df_viz['Class'] == target_class)
    ]
    fig = px.bar(
        filtered,
        x='Model',
        y='Value',
        color='Set',
        barmode='group',
        title=f"{metric} Score Comparison - Class {target_class}"
    )
    fig.update_layout(yaxis=dict(range=[0, 1]))
    fig.show()

out = widgets.interactive_output(update_plot, {
    'metric': metric_dropdown,
    'target_class': class_dropdown
})

# Display widgets and output
display(widgets.HBox([metric_dropdown, class_dropdown]), out)

Output()

## 🧪 Evaluation Summary: Downsampled Dataset (Realistic, Small Sample)

We downsampled the majority class to create a balanced dataset with 30 medicinal and 30 non-medicinal plants, then evaluated performance on a real, small test set of 16 samples (8 per class).

### Key Observations:
- Models like **Gradient Boosting** and **Random Forest** achieved modest performance (accuracy ~62% and ~56%), while **Logistic Regression** and **SVM** hovered around 50%, equivalent to random guessing.
- Extremely low test set size led to high sensitivity in evaluation — a single misclassified point shifts metrics significantly.
- The performance variability highlights the **need for more samples**, especially for the underrepresented non-medicinal class.

### Conclusion:
While this evaluation offers a more realistic view than synthetic test sets, the results are too noisy to draw strong conclusions. Further data collection or alternative modeling (e.g., regression, semi-supervised learning) may be more fruitful.